# Proyecto 1: Brecha Digital
### Preparación de datos

#### Basado en el reporte ["La Brecha Digital en California"](https://www.ppic.org/wp-content/uploads/jtf-californias-digital-divide-in-spanish.pdf)

## Pregunta(s) de Investigación:
1. ¿Qué porcentaje de hogares en el estado X tiene acceso a internte de alta velocidad?
2. ¿Varía este número según los grupos demográficos? (en este caso raza/etnia.

## Meta:
* Explorar los archivos de datos (`acsdata.data.gz`) y crear un _conjunto de datos analítico_ (un archivo derivado enfocado específicamente en análisis en mano).

## Contexto:
Obtenimos datos de la encuesta American Community Survey (ACS) de [IPUMS](https://usa.ipums.org/usa/). <br>
Contiene características demográficas:
  - edad
  - sexo
  - raza/etnia

e indicadores geográficos:
  - estado
  - condado

***

#### Paso 1: Prepara tu entorno de trabajo.

**Import**a las bibliotecas necesarias y crea objetos `Path` (ruta de archivo). Esto grarantiza reproducibilidad en distintos sistemas operativos (Windows utiliza `\` en lugar de `/` para separar los nombres de archivos.

Necesitamos: 
1. `pandas` para trabajar con los datos.
2. `pathlib`, y más específicamente su objeto `Path`, para trabajar con rutas de archivos. Esto asegura que nuestro código funcione en Windows (que utiliza `\` en sus rutas) y MacOS/Linux (los cuales utilizan `/`).
3. `datetime` - tip: Existen sistemas de control de versiones para datos pero etiquetar tus archivos de datos (cuando no son masivos) no es un mal primer paso si estás comenzando.

In [1]:
# Prepara tu entorno de trabajo
import _____ as pd
from _____ import Path
from datetime import datetime as dt
today = __.today().strftime("%d-%b-%y")

print(today)

27-Apr-19


_nota: Aunque estés utilizando Windows puedes escribir_ `/` _en tus rutas de archivo._

In [2]:
# Directorio de datos y rutas
RUTA_DATOS_EN_BRUTO = ____("../datos/brutos/")
RUTA_DATOS_________ = ____("../datos/interinos/")
RUTA_______________ = ____("../datos/procesados/")
___________________ = ____("../datos/finales/")

**NOTA:** He incluido un _script_ de `python` llamado `herramientas.py` con la función `arbol` la cual muestra el árbol de directorios. La obtuvé de el [tutorial de RealPython's sobre el módulo `pathlib`](https://realpython.com/python-pathlib/)).

    de nuestro script herramientas importa arbol para utilizarla

In [ ]:
arbol(________)

***

#### Paso 2: Carga y explora los datos

Con `pandas` _leer_ archivos de datos es tan fácil como escribir `.read_csv(RUTA_AL_ARCHIVO_CSV)` y esto es suficiente la mayoría del tiempo. La función `read_csv()` de `Pandas` es tan poderosa que incluso lee archivos comprimidos sin tener que especificar algún otro parametro. Prueba lo siguiente:

```python
datos = pd.read_csv(RUTA_DATOS_EN_BRUTO / 'acs_data.csv.gz')
datos.head()
```
_Asegurate de cambiar_ `RUTA_DE_DATOS_EN_BRUTO` _a lo que seaque hayas llamado tu variable a la que asignaste el objeto `Path` con la ruta a tus datos en bruto._

***
IPUMS ofrece algunos formatos de datos que pueden ser aún más útiles [[docs]](https://usa.ipums.org/usa-action/faq#ques12):
> Además del archivo de datos ASCII, el sistema crea un archivo de sintaxis de paquetes estadísticos para acompañar cada extracto. El archivo de sintaxis está diseñado para leer los datos ASCII mientras se aplican las variables de valor y las etiquetas apropiadas. SPSS, SAS y Stata son compatibles. Debe descargar el archivo de sintaxis con el extracto o no podrá leer los datos. El archivo de sintaxis requiere una edición menor para identificar la ubicación del archivo de datos en su computadora local.

En este caso, usaremos un archivo **Stata** (`.dta`). La razón principal es que los archivos `.dta` pueden almacenar *etiquetas de valor* que` pandas` luego puede leer y convertir columnas en columnas `Categóricas` en nuestro marco de datos de pandas. Esto 1) ahorra memoria, y 2) es una buena práctica porque ciertas ciencias sociales en serio, en serio _en serio_, y en serio _en serio_ ***en serio*** aman a Stata, por lo que sus conjuntos de datos interesantes son probablemente archivos `.dta`.

Sin embargo, `pandas` no puede leer` .dta` comprimido directamente como lo hace con los archivos `.csv`. Afortunadamente, IPUMS utiliza el formato comprimido *gzip* y `python` incluye un módulo` gzip` en su biblioteca estándar.

**Import**a gzip y prueba lo siguiente:
```python
with gzip.open(RUTA_DATOS_EN_BRUTO / 'acs_data.dta.gz') as archivo:
    datos = pd.read_stata(archivo)
```

y muestra las primeras cinco filas de tu `datos` _DataFrame_.

In [ ]:
# importa gzip y carga tus datos




In [ ]:
# muestra las primeras 5 filas


***

#### Paso 3: Familiarizate con el conjunto de datos.
Ya hemos visto `.head()` - el método de `pandas` que mostrará las primeras 5 filas de tu DataFrame. Esto te da una idea de cómo se ven sus datos. Sin embargo, hay mucho más `.info()` que puede salir de su marco de datos. También puede simplemente pedirle al DataFrame si los `.describe()`...

In [ ]:
# averigua más info de tu DataFrame
data.____()

In [ ]:
# describe tus datos
data.____()

Comprueba la 'forma' de tus datos con su atributo `.shape`. Note la falta de paréntesis.

In [ ]:
data._____

***

#### Paso 4: Recorta tus datos
En este momento estás trabajando con tu **archivo maestro**: un conjunto de datos que contiene todo lo que _podría_ necesita para su análisis. Realmente no deseas modificar este conjunto de datos porque podría estar usándolo para otros análisis. Por ejemplo, vamos a analizar el acceso a Internet de alta velocidad en un estado de tu elección, pero la próxima semana es posible que desees realizar el mismo análisis en otro estado o tal vez solo en un condado específico. Para asegurarse de que puedas **reutilizar** tus datos y tu código más adelante, creamos un _archivo de análisis_, un conjunto de datos que contiene solo los datos necesarios para **este** análisis específico a mano.

Primero, solo estamos interesados en encontrar la _ "Brecha Digital" de un estado en este momento. El **archivo maestro** contiene datos de los 50 estados y el distrito de Columbia.

Lo que quieres hacer es encontrar todas las filas donde el `statefip` coincide con el nombre de tu estado. Esto se llama indexación booleana.

Prueba lo siguiente:
```python
data['statefip'] == 'ohio'
```
_Nota: Tu puedes cambiar 'ohio' a cualquiera de los 50 estados o 'district of columbia' para DC._

In [ ]:
# Prueba indexación booleanda
___['______'] == '_______'

Esto devolverá un `pandas.Series` de booleanos (Trues y Falses) que luego puede usar para filtrar las filas innecesarias.

Es una buena práctica guardar esta _Serie_ como una variable al principio de tu código (si la conoces de antemano) o justo antes de usarla en caso de que use estas condiciones en más de un lugar. Esto te ahorrará tiempo si decide cambiar el valor que está comparando, `'ohio'` para`' california'` por ejemplo.

```python
mascara_estado = (datos['statefip'] == 'ohio')
datos[mascara_estado].head()
```

In [ ]:
# try it yourself
mask_state = (________________________ == _______)
data[mask_state].____()

let's save it to another variable with a more useful name:

```python
state_data = data[mask_state].copy()
```

You have to use `.copy()` to create actual copies of the data. If you ran
```python
state_data = data[mask_state]
```
`state_data` would be a _view_ of the `data` dataframe. This can have unintended consequences down the road if you modify your dataframes. A lot of the times you'd get just a warning and your code will run just as intented - but why take risks, right?

In [ ]:
# save your data to state_data
state_data = __________.copy()

Now, let's see what `.columns` we have in our dataframe. You can find these the same way you found the `.shape` of it earlier.

In [ ]:
state_data._____

Are there any columns that you are **confident** you don't need? If you are not 90% sure you won't need a variable don't drop it. 

Dropping columns is as easy using `.drop()` on your dataframe.

```python
state_data.drop(columns = ['list', 'of', 'columns', 'to', 'drop'])
```

If there are variables you _think_ you won't need but you're not very sure that's the case, you should explore them. 

`pandas` dataframe's columns are `pandas.Series` and they have methods and attributes just like dataframes.

Let's explore the variable `gq` which stands for `Group Quarters`. From the IPUMS [docs](https://usa.ipums.org/usa-action/variables/GQ#description_section):
>Group quarters are largely institutions and other group living arrangements, such as rooming houses and military barracks.

Let's see what `.unique()` values the `state_data['gq']` series has...

We can also see the `.value_counts()` which would give us a better idea of how useful this column might be. For example, if a column has 2 values but 99% of the observations have one value and 1% have the other - you could drop column altogether since it might not add a lot value to your analysis. 

Some variables have 100% of it's rows with the same value... \*cough\* \*cough\* `state_data['year']`...

From IPUMS [docs](https://usa.ipums.org/usa-action/variables/GQ#comparability_section):
>There are three slightly different definitions of group quarters in the IPUMS. For the period 1940-1970 (excluding the 1940 100% dataset), group quarters are housing units with five or more individuals unrelated to the householder. Before 1940 and in 1980-1990, units with 10 or more individuals unrelated to the householder are considered group quarters. **In the 2000 census, 2010 census, the ACS and the PRCS, no threshold was applied; for a household to be considered group quarters, it had to be on a list of group quarters that is continuously maintained by the Census Bureau. In earlier years, a similar list was used, with the unrelated-persons rule imposed as a safeguard.**

Because of this and the fact that most of our observations fall into the 1970 and 1990 definition, we'll stick to those 2 for our analysis.

Let's create another _mask_ to filter out households that don't fit our definition.

For multiple conditions we use: `&` and `|` operators (**and** and **or**, respectively)

In [ ]:
mask_household = ( CONDITION ONE ) | ( CONDITION TWO )

**note**: another value added from having categorical variables is that, **if** they are ordered, you can use the `<`, `>` operators for conditions as well.
```python
mask_household = (state_data['gq'] <= 'additional households under 1990 definition')
```

_note: since you are overwriting_ `state_data` _you don't need to use_ `.copy()` _but it doesn't hurt and if you're a beginner at_ `pandas` _it's good practice for when you actually need to use_ `.copy()`.

In [ ]:
state_data = state_data[mask_household].______()

At this point you're really close to a `working_data` dataset. You have:
1. Kept one state's information and dropped the rest.
2. Kept only those _households_ you're interested in and dropped the rest

Our research question 1 is: "What share of households in X state have access to high-speed internet?"

Mathematically, 
$$ \frac{households\ with\ high\ speed\ internet}{households\ in\ state}$$

Your `state_data` dataset contains all you need to find the answer. 

***

#### Step 5: Save your data

Now that you have trimmed your **masterfile** into a `working_data` dataset you should save it. 

We've been working with a `.dta` file and it'd be best if we keep it that way. 

Try the following:
```python
state_data.to_stata(INTERIM_DATA_PATH / f'state_data-{today}.dta', write_index = False)
```

A few things:
1. We're using `f-strings` to tag our datafile with today's date.
2. You're turning off the `write_index` flag so you don't add a 'index' column to your `.dta` file. In this dataset, our index isn't meaningful. In other analysis you might have a meaningful index and you won't want to turn off this flag.

***

#### Step 6: Bonus
What if we changed our research question a little bit, from <br>_"What share of households in X state have access to high-speed internet?_ <br>to <br>_"What share of households **with school-age children** in X state have access to high-speed internet?"_

This would be an interesting statistic to policy-makers, especially if we find discrepancies across demographic groups (research question 2).

The challenge here is that the **unit of observation** in our `state_data` file is a (weighted) person and we want to _filter_ out those **households** without any school-age children in them. This might sound a little complicated at first but it just requires modifying our previous workflow just a little.

We need to do a few things:
1. Define what we mean by school-age children.
2. Create a _mask_ to grab all households where these children are.
3. Create a list of unduplicated household identifiers (`'serial'`) 
4. Use that list to drop unwanted observations.

#### Step 6.1: School-age children

Most people would agree school age (Elementary through High School) is 6 - 17 year olds. Some people are interested in K-12 (5 - 17 or 18). Some people wouldn't include 18 year olds. Whatever measure you choose you must be able to defend why you are choosing it. 

For this analysis, I'll suggest we use 5 - 18 year olds (K-12) but you can choose whatever age range you want. Maybe high-school kids 14-18? That'd be interesting, you probably need access to high-speed internet at home a lot more in high school than you do in kindergarden. 

In [ ]:
mask_children = (state_data['age'] >= ___) & (___________________ <= )

<summary> <i>What data type is</i> <span style='font-family:monospace'>state_data['age']</span> <i>again?</i> 
    <details> 
        Categorical. This means that you even though its values _look_ like numbers, they're actually _value labels_ aka strings.
    </details>
</summary>

Now that we have our _mask_, we can use it to create a list of households with children in them.

Earlier we applied a mask to a dataframe and saved it to another variable. Here, we'll go a step further and grab just a column of that _filtered out_ dataframe.

Try it yourself first.

*Hint: How did we grab and explore a single column of a dataframe earlier?*

In [ ]:
households_with_children = _________________________________________

In [ ]:
households_with_children.head()

How do you think we can `.drop_duplicates()`?

Once you have your unduplicated list of households with children all you have to do is to check if a `serial` value from our `state_data` dataset `.isin()` our `households_with_children` series.

Let's save that as our `working_data` dataset and save that to memory.

In [ ]:
working_data = _____________________

```python
working_data.to_stata(INTERIM_DATA_PATH / f'working_data-{today}.dta', write_index = False)
```